# Artificial Intelligence Nanodegree
## Machine Translation 

- **Preprocess** - Convert text to sequence of integers.
- **Models** - Create models which accepts a sequence of integers as input and returns a probability distribution over possible translations. 
- **Prediction** Run the model on English text.

In [1]:
%load_ext autoreload
%aimport helper, tests
%autoreload 1

In [2]:
import collections

import helper
import numpy as np
import project_tests as tests

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, Sequential
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector 
from keras.layers import Bidirectional, Dropout 
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy
from keras.layers.embeddings import Embedding
from keras.layers import LSTM

Using TensorFlow backend.


In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 9579979678650389834
, name: "/gpu:0"
device_type: "GPU"
memory_limit: 357433344
locality {
  bus_id: 1
}
incarnation: 12965914988060449339
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0"
]


## Dataset
We begin by investigating the dataset that will be used to train and evaluate your pipeline.  The most common datasets used for machine translation are from [WMT](http://www.statmt.org/).  However, that will take a long time to train a neural network on.  We'll be using a dataset we created for this project that contains a small vocabulary.  You'll be able to train your model in a reasonable time with this dataset.
### Load Data
The data is located in `data/small_vocab_en` and `data/small_vocab_fr`. The `small_vocab_en` file contains English sentences with their French translations in the `small_vocab_fr` file. Load the English and French data from these files from running the cell below.

In [4]:
# Load English data
english_sentences = helper.load_data('data/small_vocab_en')
# Load French data
french_sentences = helper.load_data('data/small_vocab_fr')

print('Dataset Loaded')

Dataset Loaded


### Files
Each line in `small_vocab_en` contains an English sentence with the respective translation in each line of `small_vocab_fr`.  View the first two lines from each file.

In [5]:
for sample_i in range(5):
    print('small_vocab_en Line {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('small_vocab_fr Line {}:  {}'.format(sample_i + 1, french_sentences[sample_i]))

small_vocab_en Line 1:  new jersey is sometimes quiet during autumn , and it is snowy in april .
small_vocab_fr Line 1:  new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
small_vocab_en Line 2:  the united states is usually chilly during july , and it is usually freezing in november .
small_vocab_fr Line 2:  les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .
small_vocab_en Line 3:  california is usually quiet during march , and it is usually hot in june .
small_vocab_fr Line 3:  california est généralement calme en mars , et il est généralement chaud en juin .
small_vocab_en Line 4:  the united states is sometimes mild during june , and it is cold in september .
small_vocab_fr Line 4:  les états-unis est parfois légère en juin , et il fait froid en septembre .
small_vocab_en Line 5:  your least liked fruit is the grape , but my least liked is the apple .
small_vocab_fr Line 5:  votre moins aimé fruit est le raisin , m

### Vocabulary
The complexity of the problem is determined by the complexity of the vocabulary.  A more complex vocabulary is a more complex problem.  Let's look at the complexity of the dataset we'll be working with.

In [7]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

1823250 English words.
227 unique English words.
10 Most common words in the English dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

1961295 French words.
355 unique French words.
10 Most common words in the French dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"


For this project, i'll convert the text into sequences of integers using the following preprocess methods:

1. Tokenize the words into ids
2. Add padding to make all the sequences the same length.

Time to start preprocessing the data...
### Tokenize (IMPLEMENTATION)
For a neural network to predict on text data, it first has to be turned into data it can understand. Text data like "dog" is a sequence of ASCII character encodings.  Since a neural network is a series of multiplication and addition operations, the input data needs to be number(s).

We can turn each character into a number or each word into a number.  These are called character and word ids, respectively.  Character ids are used for character level models that generate text predictions for each character.  A word level model uses word ids that generate text predictions for each word.  Word level models tend to learn better, since they are lower in complexity, so we'll use those.

In [8]:
def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    
    #x_tk = Tokenizer(char_level=True)
    x_tk = Tokenizer()
    x_tk.fit_on_texts(x)
    
    return x_tk.texts_to_sequences(x), x_tk

tests.test_tokenize(tokenize)

# Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'the': 1, 'quick': 2, 'a': 3, 'brown': 4, 'fox': 5, 'jumps': 6, 'over': 7, 'lazy': 8, 'dog': 9, 'by': 10, 'jove': 11, 'my': 12, 'study': 13, 'of': 14, 'lexicography': 15, 'won': 16, 'prize': 17, 'this': 18, 'is': 19, 'short': 20, 'sentence': 21}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [18, 19, 3, 20, 21]


### Padding (IMPLEMENTATION)
When batching the sequence of word ids together, each sequence needs to be the same length.  Since sentences are dynamic in length, we can add padding to the end of the sequences to make them the same length.

Make sure all the English sequences have the same length and all the French sequences have the same length by adding padding to the **end** of each sequence using Keras's [`pad_sequences`](https://keras.io/preprocessing/sequence/#pad_sequences) function.

In [9]:
def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    
    if length is None:
        length = max([len(sentence) for sentence in x])
    return pad_sequences(x, maxlen=length, truncating='post', padding='post')

tests.test_pad(pad)

# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [1 2 4 5 6 7 1 8 9]
  Output: [1 2 4 5 6 7 1 8 9 0]
Sequence 2 in x
  Input:  [10 11 12  2 13 14 15 16  3 17]
  Output: [10 11 12  2 13 14 15 16  3 17]
Sequence 3 in x
  Input:  [18 19  3 20 21]
  Output: [18 19  3 20 21  0  0  0  0  0]


### Preprocess Pipeline
Your focus for this project is to build neural network architecture, so we won't ask you to create a preprocess pipeline.  Instead, we've provided you with the implementation of the `preprocess` function.

In [10]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)
    
    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)
    
    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)
    
max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)

print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)

Data Preprocessed
Max English sentence length: 15
Max French sentence length: 21
English vocabulary size: 199
French vocabulary size: 344


## Models
In this section, you will experiment with various neural network architectures.
You will begin by training four relatively simple architectures.
- Model 1 is a simple RNN
- Model 2 is a RNN with Embedding
- Model 3 is a Bidirectional RNN
- Model 4 is an optional Encoder-Decoder RNN

After experimenting with the four simple architectures, you will construct a deeper architecture that is designed to outperform all four models.
### Ids Back to Text
The neural network will be translating the input to words ids, which isn't the final form we want.  We want the French translation.  The function `logits_to_text` will bridge the gab between the logits from the neural network to the French translation.  You'll be using this function to better understand the output of the neural network.

In [11]:
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


### Model 1: RNN (IMPLEMENTATION)
![RNN](images/rnn.png)
A basic RNN model is a good baseline for sequence data.  In this model, you'll build a RNN that translates English to French.

## Model Parameters 

In [41]:
# Shapes: english, french and output sentences
print("english_sentences, shape: ", preproc_english_sentences.shape)
print("french_sentences, shape: ", preproc_french_sentences.shape)
print('output sequence, shape: ', preproc_french_sentences.shape[1])
# The output is based in french sentences because is the target language

english_sentences, shape:  (137861, 15)
french_sentences, shape:  (137861, 21, 1)
output sequence, shape:  21


In [42]:
# Lenght of vocabulary's
print("english vocab size: ", len(english_tokenizer.word_index))
print("french vocab size: ", len(french_tokenizer.word_index))

english vocab size:  199
french vocab size:  344


In [43]:
# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Input shape of simple RNN
print(tmp_x.shape)

(137861, 21, 1)


## 1. Simple Model RNN

In [44]:
def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # Build the layers
    learning_rate = 0.01
    
    
    model = Sequential()
    
    # input
    model.add(GRU(128, 
                  dropout = 0.2,
                  input_shape = input_shape[1:], 
                  return_sequences=True))
    
    # output
    model.add(TimeDistributed(Dense(french_vocab_size, 
                                    activation='softmax')))
    
    # Compile model
    model.compile(loss = sparse_categorical_crossentropy,
                  optimizer = Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model

tests.test_simple_model(simple_model)

In [45]:
# Train the neural network
simple_rnn_model = simple_model(tmp_x.shape,
                                max_french_sequence_length,
                                english_vocab_size,
                                french_vocab_size)
# Watch Model
print("Resume Model:")
print(simple_rnn_model.summary())

simple_rnn_model.fit(tmp_x, 
                     preproc_french_sentences, 
                     batch_size = 1024, 
                     epochs = 10, 
                     validation_split = 0.2)

Resume Model:
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_24 (GRU)                 (None, 21, 128)           49920     
_________________________________________________________________
time_distributed_24 (TimeDis (None, 21, 344)           44376     
Total params: 94,296
Trainable params: 94,296
Non-trainable params: 0
_________________________________________________________________
None
Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 10s 93us/step - loss: 2.0835 - acc: 0.5359 - val_loss: nan - val_acc: 0.5885
Epoch 2/10
110288/110288 [==============================] - 8s 69us/step - loss: 1.4162 - acc: 0.6166 - val_loss: nan - val_acc: 0.5790
Epoch 3/10
110288/110288 [==============================] - 8s 68us/step - loss: 1.2728 - acc: 0.6378 - val_loss: nan - val_acc: 0.5731
Epoch 4/10
110288/110288 [=============================

In [46]:
# Print prediction(s)
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], 
                                              french_tokenizer))

print("\nCorrect Translation:")
print(french_sentences[:1])

print("\nOriginal text:")
print(english_sentences[:1])

new jersey est parfois agréable en juin mais mais mais agréable agréable agréable <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

Correct Translation:
["new jersey est parfois calme pendant l' automne , et il est neigeux en avril ."]

Original text:
['new jersey is sometimes quiet during autumn , and it is snowy in april .']


In [47]:
# Score accuracy
score = simple_rnn_model.evaluate(tmp_x, 
                                  preproc_french_sentences, 
                                  verbose = 0)
print("Score accuracy: ", score[1])

Score accuracy:  0.546695265517


### Model 2: Embedding (IMPLEMENTATION)

## 2. Embedding Model

In [54]:
def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a RNN model using word embedding on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # Implement
    learning_rate = 0.006
    
    model = Sequential()
    
    # input
    model.add(Embedding(max(english_vocab_size, 
                            french_vocab_size),
              128,
              input_length = output_sequence_length))
    
    model.add(LSTM(128, 
                   dropout = 0.1, 
                   return_sequences = True))
    
    # output
    model.add(Dense(french_vocab_size, 
                    activation = 'softmax'))
    
    # Compile model
    model.compile(loss = sparse_categorical_crossentropy,
                  optimizer = Adam(learning_rate),
                  metrics = ['accuracy'])
    return model

tests.test_embed_model(embed_model)

In [55]:
# Reshape the input
tmp_x = pad(preproc_english_sentences,
            preproc_french_sentences.shape[1])

print(tmp_x.shape)

(137861, 21)


In [56]:
# Train the neural network
embed_model_rnn = embed_model(tmp_x.shape,
                              preproc_french_sentences.shape[1],
                              len(english_tokenizer.word_index),
                              len(french_tokenizer.word_index))

# Watch Model
print("Resume Model:")
print(embed_model_rnn.summary())

embed_model_rnn.fit(tmp_x,
                    preproc_french_sentences,
                    batch_size = 256,
                    epochs = 10,
                    validation_split = 0.2)

Resume Model:
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_17 (Embedding)     (None, 21, 128)           44032     
_________________________________________________________________
lstm_17 (LSTM)               (None, 21, 128)           131584    
_________________________________________________________________
dense_30 (Dense)             (None, 21, 344)           44376     
Total params: 219,992
Trainable params: 219,992
Non-trainable params: 0
_________________________________________________________________
None
Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 26s 235us/step - loss: 1.1084 - acc: 0.7358 - val_loss: nan - val_acc: 0.8725
Epoch 2/10
110288/110288 [==============================] - 23s 204us/step - loss: 0.3423 - acc: 0.8905 - val_loss: nan - val_acc: 0.9075
Epoch 3/10
110288/110288 [============================

In [57]:
# Print prediction(s)
print(logits_to_text(embed_model_rnn.predict(tmp_x[:1])[0], 
                                             french_tokenizer))

print("\nCorrect Translation:")
print(french_sentences[:1])

print("\nOriginal text:")
print(english_sentences[:1])

new jersey est parfois calme en l' automne et il est neigeux en avril <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

Correct Translation:
["new jersey est parfois calme pendant l' automne , et il est neigeux en avril ."]

Original text:
['new jersey is sometimes quiet during autumn , and it is snowy in april .']


In [58]:
# Score accuracy
score = embed_model_rnn.evaluate(tmp_x, 
                                 preproc_french_sentences, 
                                 verbose = 0)
print("Score accuracy: ", score[1])

Score accuracy:  0.932364287643


### Model 3: Bidirectional RNNs (IMPLEMENTATION)

One restriction of a RNN is that it can't see the future input, only the past.  This is where bidirectional recurrent neural networks come in.  They are able to see the future data.

## 3. Bidirectional RNNs Model

In [59]:
# Reshaping the input to work with Bidirectional RNNs Model
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))


# Input shape of Bidirectional RNNs Model
print(tmp_x.shape)

(137861, 21, 1)


In [60]:
def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a bidirectional RNN model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
        
    # Implement
    learning_rate = 0.008
        
    model = Sequential()
    
    # input
    model.add(Bidirectional(GRU(128, 
                                return_sequences = True), 
                            input_shape = input_shape[1:]))
       
    model.add(TimeDistributed(Dense(1024, 
                                    activation = 'relu')))
    
    model.add(Dropout(0.1))
    
    # output
    model.add(TimeDistributed(Dense(french_vocab_size,
                                    activation = 'softmax')))
    


    # Compile model
    model.compile(loss = sparse_categorical_crossentropy,
                  optimizer = Adam(learning_rate),
                  metrics = ['accuracy'])
    
                                 
    return model

tests.test_bd_model(bd_model)

In [64]:
# Train and Print prediction(s)
# Train the neural network
bd_model_rnn = bd_model(tmp_x.shape,
                        max_french_sequence_length,
                        english_vocab_size+1,
                        french_vocab_size+1)

# Watch Model
print("Resume Model:")
print(bd_model_rnn.summary())

bd_model_rnn.fit(tmp_x, 
                 preproc_french_sentences, 
                 batch_size = 256, 
                 epochs = 10, 
                 validation_split = 0.2)

Resume Model:
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_25 (Bidirectio (None, 21, 256)           99840     
_________________________________________________________________
time_distributed_29 (TimeDis (None, 21, 1024)          263168    
_________________________________________________________________
dropout_14 (Dropout)         (None, 21, 1024)          0         
_________________________________________________________________
time_distributed_30 (TimeDis (None, 21, 345)           353625    
Total params: 716,633
Trainable params: 716,633
Non-trainable params: 0
_________________________________________________________________
None
Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 37s 334us/step - loss: 1.2361 - acc: 0.6484 - val_loss: 0.9130 - val_acc: 0.7004
Epoch 2/10
110288/110288 [==============================]

In [65]:
# Print prediction(s)
print(logits_to_text(bd_model_rnn.predict(tmp_x[:1])[0], 
                                          french_tokenizer))

print("\nCorrect Translation:")
print(french_sentences[:1])

print("\nOriginal text:")
print(english_sentences[:1])

new jersey est parfois calme en cours et l' est neigeux en en <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

Correct Translation:
["new jersey est parfois calme pendant l' automne , et il est neigeux en avril ."]

Original text:
['new jersey is sometimes quiet during autumn , and it is snowy in april .']


In [66]:
# Score accuracy
score = bd_model_rnn.evaluate(tmp_x, 
                              preproc_french_sentences, 
                              verbose = 0)
print("Score accuracy: ", score[1])

Score accuracy:  0.853862160226


### Model 4: Encoder-Decoder 
Time to look at encoder-decoder models.  This model is made up of an encoder and decoder. The encoder creates a matrix representation of the sentence.  The decoder takes this matrix as input and predicts the translation as output.

In [80]:
def encdec_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train an encoder-decoder model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # Implement
    learning_rate = 0.008
    
    model = Sequential()
    
    # encoder
    model.add(GRU(128, input_shape = input_shape[1:], go_backwards=True))
    
    model.add(RepeatVector(output_sequence_length))
    
    
    # decoder
    model.add(GRU(128,
                  return_sequences = True))
    
    model.add(TimeDistributed(Dense(1024, activation='relu')))
    model.add(Dropout(0.2))
    
    
    # output
    model.add(TimeDistributed(Dense(french_vocab_size,
                                    activation = 'softmax')))
    
    # Compile model
    model.compile(loss = sparse_categorical_crossentropy,
                  optimizer = Adam(learning_rate),
                  metrics = ['accuracy'])
    return model

tests.test_encdec_model(encdec_model)

In [81]:
# Reshape input for basic RNN
# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences)
tmp_x = tmp_x.reshape((-1, preproc_english_sentences.shape[1], 1))

# Input shape of simple RNN
print(tmp_x.shape)

(137861, 15, 1)


In [82]:
# OPTIONAL: Train and Print prediction(s)
# Train the neural network
encdec_model_rnn = encdec_model(tmp_x.shape,
                                max_french_sequence_length,
                                english_vocab_size+1,
                                french_vocab_size+1)

# Watch Model
print("Resume Model:")
print(encdec_model_rnn.summary())

encdec_model_rnn.fit(tmp_x, 
                     preproc_french_sentences, 
                     batch_size = 256, 
                     epochs = 10, 
                     validation_split = 0.2)

Resume Model:
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_42 (GRU)                 (None, 128)               49920     
_________________________________________________________________
repeat_vector_19 (RepeatVect (None, 21, 128)           0         
_________________________________________________________________
gru_43 (GRU)                 (None, 21, 128)           98688     
_________________________________________________________________
time_distributed_45 (TimeDis (None, 21, 1024)          132096    
_________________________________________________________________
dropout_22 (Dropout)         (None, 21, 1024)          0         
_________________________________________________________________
time_distributed_46 (TimeDis (None, 21, 345)           353625    
Total params: 634,329
Trainable params: 634,329
Non-trainable params: 0
________________________________________________________

In [83]:
# Print prediction(s)
print(logits_to_text(encdec_model_rnn.predict(tmp_x[:1])[0], 
                                              french_tokenizer))

print("\nCorrect Translation:")
print(french_sentences[:1])

print("\nOriginal text:")
print(english_sentences[:1])

new jersey est parfois parfois au mois de il il est est en <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

Correct Translation:
["new jersey est parfois calme pendant l' automne , et il est neigeux en avril ."]

Original text:
['new jersey is sometimes quiet during autumn , and it is snowy in april .']


In [84]:
# Score accuracy
score = encdec_model_rnn.evaluate(tmp_x, 
                                  preproc_french_sentences, 
                                  verbose = 0)
print("Score accuracy: ", score[1])

Score accuracy:  0.730897378037


### Model 5: Custom 
Model that incorporates embedding and a bidirectional rnn into one model.

In [35]:
def model_final(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # Implement
    
    learning_rate = 0.006
    
    model = Sequential()
    
    # Embedding -------------------------------------------
    
    # input
    model.add(Embedding(english_vocab_size,
                        128,
                        input_length = input_shape[1:],
                        input_shape = input_shape[1:]))
                      
    model.add(LSTM(128, 
                   dropout = 0.1, 
                   return_sequences = True))
    
    
    # Bidirectional --------------------------------------
    model.add(Bidirectional(GRU(128)))
    
    # Encoder --------------------------------------------
    model.add(RepeatVector(output_sequence_length))

        
    # Bidirectional --------------------------------------
    model.add(Bidirectional(GRU(128, return_sequences=True)))
    
    # Decoder --------------------------------------------    
    model.add(TimeDistributed(Dense(256, activation='relu')))
    model.add(Dropout(0.2))
    
    # output ---------------------------------------------
    model.add(TimeDistributed(Dense(french_vocab_size, 
                                    activation = 'softmax')))
    
    # Compile model
    model.compile(loss = sparse_categorical_crossentropy,
                  optimizer = Adam(learning_rate),
                  metrics = ['accuracy'])
    
    return model


tests.test_model_final(model_final)

In [37]:
print('Final Model Loaded')
# Train the final model
model_final_rnn = model_final(preproc_english_sentences.shape,
                              preproc_french_sentences.shape[1],
                              len(english_tokenizer.word_index)+1,
                              len(french_tokenizer.word_index)+1)

# Watch Model
print("Resume Model:")
print(model_final_rnn.summary())

model_final_rnn.fit(preproc_english_sentences,
                    preproc_french_sentences,
                    batch_size = 256,
                    epochs = 10,
                    validation_split = 0.2)

Final Model Loaded
Resume Model:
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (None, 15, 128)           25600     
_________________________________________________________________
lstm_11 (LSTM)               (None, 15, 128)           131584    
_________________________________________________________________
bidirectional_21 (Bidirectio (None, 256)               197376    
_________________________________________________________________
repeat_vector_11 (RepeatVect (None, 21, 256)           0         
_________________________________________________________________
bidirectional_22 (Bidirectio (None, 21, 256)           295680    
_________________________________________________________________
time_distributed_21 (TimeDis (None, 21, 256)           65792     
_________________________________________________________________
dropout_11 (Dropout)         (None, 21, 256

In [38]:
# Print prediction(s)
print(logits_to_text(model_final_rnn.predict(preproc_english_sentences[:1])[0], 
                                              french_tokenizer))

print("\nCorrect Translation:")
print(french_sentences[:1])

print("\nOriginal text:")
print(english_sentences[:1])

new jersey est parfois calme pendant l' automne et il est neigeux en avril <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

Correct Translation:
["new jersey est parfois calme pendant l' automne , et il est neigeux en avril ."]

Original text:
['new jersey is sometimes quiet during autumn , and it is snowy in april .']


In [39]:
# Score accuracy
score = model_final_rnn.evaluate(preproc_english_sentences, 
                                 preproc_french_sentences, 
                                 verbose = 0)
print("Score accuracy: ", score[1])

Score accuracy:  0.97106679631


## Prediction (IMPLEMENTATION)

In [40]:
def final_predictions(x, y, x_tk, y_tk):
    """
    Gets predictions using the final model
    :param x: Preprocessed English data
    :param y: Preprocessed French data
    :param x_tk: English tokenizer
    :param y_tk: French tokenizer
    """
    # ---- Done Up -----
    # Train neural network using model_final
    # model = None
    # ---- Done Up ----- 
    
    ## DON'T EDIT ANYTHING BELOW THIS LINE
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'

    sentence = 'he saw a old yellow truck'
    sentence = [x_tk.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
    sentences = np.array([sentence[0], x[0]])
    predictions = model_final_rnn.predict(sentences, len(sentences))

    print('Sample 1:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    print('Il a vu un vieux camion jaune')
    print('Sample 2:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
    print(' '.join([y_id_to_word[np.max(x)] for x in y[0]]))


final_predictions(preproc_english_sentences, 
                  preproc_french_sentences, 
                  english_tokenizer, 
                  french_tokenizer)

Sample 1:
il a vu un vieux camion jaune <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
Il a vu un vieux camion jaune
Sample 2:
new jersey est parfois calme pendant l' automne et il est neigeux en avril <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
new jersey est parfois calme pendant l' automne et il est neigeux en avril <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
